In [206]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import time

torch.manual_seed(1)

In [186]:
with open(r'C:\Users\nelso\OneDrive\Machine_Learning\Project\tweets.txt', encoding = 'utf8') as f:
    tweets = f.readlines()

del tweets[0]

In [187]:
with open(r'C:\Users\nelso\OneDrive\Machine_Learning\Project\tweets.txt', encoding = 'utf8') as f:
    content=list(set(f.read()))

In [208]:
ix_to_char = {ix:char for ix, char in enumerate(content)}
char_to_ix = {char:ix for ix, char in enumerate(content)}
vocab_size = len(char_to_ix)

In [216]:
def prepare_tweet(tweet, target = False):
    if target==True:
        tweet_ix = torch.tensor([char_to_ix[c] for c in tweet[1:]], dtype=torch.long)
    else:
        tweet_ix = torch.tensor([char_to_ix[c] for c in tweet[:-1]], dtype=torch.long)
    return tweet_ix

In [212]:
class RNN(nn.Module):
    def __init__(self, embed_dim, hidden_dim, vocab_size, n_layers=1):
        super(RNN, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embed = nn.Embedding(vocab_size, embed_dim)
        
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers = n_layers)
        
        self.hidden2char = nn.Linear(hidden_dim, vocab_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return(torch.zeros(self.n_layers, 1, self.hidden_dim),
              torch.zeros(self.n_layers, 1, self.hidden_dim))
    
    def forward(self, tweet):
        embeds = self.embed(tweet)
        lstm_out, self.hidden = self.lstm(embeds.view(len(tweet), 1, -1),
                                         self.hidden)
        output = self.hidden2char(lstm_out.view(len(tweet), -1))
        log_probs = F.log_softmax(output, dim=1)
        return log_probs, hidden
        

In [272]:
hidden_dim = 128
embed_size = 128
epochs = 3

model = RNN(embed_size, hidden_dim, vocab_size)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=.1)

start = time.time()

for epoch in range(epochs):
    total_loss = 0
    for tweet in tweets:
        model.zero_grad()
        
        model.hidden = model.init_hidden()
        
        tweet_x = prepare_tweet(tweet)
        tweet_y = prepare_tweet(tweet, target=True)
        
        log_probs = model(tweet_x)
        
        loss = loss_function(log_probs, tweet_y)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    print('Epoch', epoch+1, 'Completed in %.0f' %(time.time()-start),'seconds - Loss: %.2f' %total_loss)

Epoch 1 Completed in 44 seconds - Loss: 1099.67
Epoch 2 Completed in 89 seconds - Loss: 915.93
Epoch 3 Completed in 129 seconds - Loss: 849.78


In [270]:
def generate_tweet(first):
    with torch.no_grad():
        char = torch.tensor(char_to_ix[first], dtype = torch.long)
        hidden = model.init_hidden()
        output_tweet = first
        letter = first
    
        #while letter != '/n':
        for i in range(5):
            output = model(char)
            topv, topi = output.topk(1)
            topi = topi[0][0].item()
            letter = ix_to_char[topi]
            output_tweet += letter
            char = torch.tensor(char_to_ix[letter], dtype = torch.long)
        
    return output_tweet

In [271]:
generate_tweet('a')

RuntimeError: dimension specified as 1 but tensor has no dimensions